## Import Libraries

In [206]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import pickle

## Data Preparation

In [235]:
def file_to_dataFrame(file_name, subset=True, nrows=1000000):
    if subset:
        df = pd.read_csv(file_name, nrows=nrows, parse_dates=['pickup_datetime'])
    else:
        df = pd.read_csv(file_name, parse_dates=['pickup_datetime'])
    return df

In [208]:
def make_Xtest_ytest(df):
    y_test = df['key']
    y_test = pd.DataFrame(y_test)
    X_test = df.drop('key', axis=1)
    return X_test, y_test

## Clean Data

In [209]:
def clean_data(df):
    print('dropping nan:', len(df))
    df = df.dropna(axis=0, subset=['dropoff_latitude'])
    df = df.drop('key', axis=1)
    print('nan dropped:', len(df))
    return df

In [210]:
def lat_lon_US(df):
    # Choose cab rides whose pickup and dropoff are the US Mainland
    # Declare constants
    latmin = 5.496100
    latmax = 71.538800
    longmin = -124.482003
    longmax = -66.885417

    # Create dataframe with correct coordinates
    df = df[((((df['pickup_longitude']<=longmax) & (df['pickup_longitude']>=longmin)) & ((df['pickup_latitude']<=latmax) & (df['pickup_latitude']>=latmin)))) & ((((df['dropoff_longitude']<=longmax) & (df['dropoff_longitude']>=longmin)) & ((df['dropoff_latitude']<=latmax) & (df['dropoff_latitude']>=latmin))))]
    
    print('US Mainland Only dropped:', len(df))

    return df

In [211]:
def lat_lon_NYC(df):
    # Find cab rides whose pickup or dropoff are within NYC boundaries
    # Declare constants
    latmin = 40.477399
    latmax = 40.917577
    longmin = -74.259090
    longmax = -73.700272

    # Create dataframe with correct coordinates
    df = df[((((df['pickup_longitude']<=longmax) & (df['pickup_longitude']>=longmin)) & ((df['pickup_latitude']<=latmax) & (df['pickup_latitude']>=latmin)))) | ((((df['dropoff_longitude']<=longmax) % (df['dropoff_longitude']>=longmin)) & ((df['dropoff_latitude']<=latmax) & (df['dropoff_latitude']>=latmin))))]
    
    print('NYC Taxis Only:', len(df))

    return df

In [212]:
def max_Riders(df, num=7):
    # Only choose cabs between 1 and num riders
    df = df[(df['passenger_count'] <= num) & (df['passenger_count'] > 0)]
    print('Max Passengers 7:', len(df))
    return df

In [213]:
def add_distance(df):

    # Define coordinates (x,y)
    x1 = df['pickup_latitude']
    y1 = df['pickup_longitude']
    x2 = df['dropoff_latitude']
    y2 = df['dropoff_longitude']

    # Create Euclidean Distrance column
    df['euclidean_distance'] = np.sqrt((y2-y1)**2 + (x2-x1)**2)

    # Create Taxicab Distance column
    df['taxicab_distance'] = np.abs(y2-y1) + np.abs(x2-x1)

    # Convert to miles
    df['euclidean_distance'] = df['euclidean_distance'] * 69
    df['taxicab_distance'] = df['taxicab_distance'] * 69
    
    print('Distance Columns added...')

    return df

In [214]:
def min_Fare(df):
    # Eliminate unrealistic plots
    df = df[df['fare_amount'] >= (df['euclidean_distance'] * 2 + 2.5)]
    print('Min fares dropped:', len(df))

    return df

In [215]:
def max_Fare(df):
    df = df[(df['fare_amount'] <= (df['taxicab_distance'] * 48 + 16)) | (df['fare_amount'] <= 56)]
    print('Max fares dropped:', len(df))
    return df

In [216]:
def no_distance(df):
    # Elminate fares that traveled no distance
    df = df[df['euclidean_distance']>0]
    print('No distance dropped:', len(df))
    return df

In [217]:
def row_elimination(df):
    df = clean_data(df)
    df = lat_lon_US(df)
    df = lat_lon_NYC(df)
    df = max_Riders(df)
    df = add_distance(df)
    df = min_Fare(df)
    df = max_Fare(df)
    df = no_distance(df)
    return df

## X_train, y_train Columns

In [218]:
def make_X_y(df):
    X = df.drop('fare_amount', axis=1)
    y = df['fare_amount'].copy()
    return X,y

## Garbage Removal

In [236]:
# Get rid of accumulated garbage
import gc
gc.collect()

204

## Add Attributes

### Time

In [220]:
def add_Time_units(df):
    
    df['month'] = df['pickup_datetime'].dt.month
    df['year'] = df['pickup_datetime'].dt.year
    df['hour'] = df['pickup_datetime'].dt.hour
    df['minute'] = df['pickup_datetime'].dt.minute
    df['second'] = df['pickup_datetime'].dt.second
    df['dayofweek'] = df['pickup_datetime'].dt.dayofweek
    
    from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
    dr = pd.date_range(start='2009-01-01', end='2015-12-31')
    cal = calendar()
    holidays = cal.holidays(start=dr.min(), end=dr.max())
    df['holiday'] = df['pickup_datetime'].dt.date.astype('datetime64').isin(holidays)
    
    df = df.drop('pickup_datetime', axis=1)

    df['15_min_intervals'] = 4 * df['hour'] + (df['minute']/15).astype(int)
    df['total_seconds'] = 3600 * df['hour'] + 60 * df['minute'] + df['second']
        
    return df

In [221]:
def add_Time_columns(df):

    def summer_month(row):
        if row['month'] in [6,7,8]:
            return 1
        else:
            return 0
    
    df['summer_month'] = df.apply(summer_month, axis=1)
    
    
    def cold_month(row):
        if row['month'] in [1,2,3,11,12]:
            return 1
        else:
            return 0
    
    df['cold_month'] = df.apply(cold_month, axis=1)
    
    def weekend(row):
        if (row['dayofweek'] in [5,6]) | (row['holiday']):
            return 1
        else:
            return 0

    df['weekend'] = df.apply(weekend, axis=1)

    def rush_hour(row):
        if ((row['hour'] in [7,8,9,15,16,17,18,19]) & (row['weekend'] == 0)) & (not row['holiday']):
            return 1
        else:
            return 0

    df['rush_hour'] = df.apply(rush_hour, axis=1)
    
    def morning_rush(row):
        if ((row['hour'] in [6,7,8,9]) & (row['weekend'] == 0)) & (not row['holiday']):
            return 1
        else:
            return 0

    df['morning_rush'] = df.apply(morning_rush, axis=1)

    def night_rush(row):
        if (row['hour'] in [19,20,21,22,23,24,1]) & (row['dayofweek'] in [3,4,5]):
            return 1
        else:
            return 0
    
    df['night_rush'] = df.apply(night_rush, axis=1)

    def night_charge(row):
        if row['hour'] in [20,21,22,23,24,1,2,3,4,5,6]:
            return 1
        else:
            return 0

    df['night_charge'] = df.apply(night_charge, axis=1)

    def weekday_surcharge(row):
        if ((row['hour'] in [16,17,18,19,20]) & (row['dayofweek'] in [0,1,2,3,4])) & (not row['holiday']):
            return 1
        else:
            return 0

    df['weekday_surcharge'] = df.apply(weekday_surcharge, axis=1)
        
    return df

In [222]:
def add_Time(df):
    df = add_Time_units(df)
    df = add_Time_columns(df)
    return df

### Manhattan

In [223]:
# Define line from two points and a provided column
def two_points_line(a, b, column):
        
    # Case when y-values are the same
    if b[1]==a[1]:
        
        # Slope defaults to 0
        slope = 0
        
    # Case when x-values are the same
    elif b[0]==a[0]:
        
        # Case when max value is less than 999999999
        if column.max() < 999999999:
            
            # Add 999999999 to max value
            slope = column.max() + 999999999
        
        # All other cases
        else:
            
            # Multiply max value by itself (greater than 999999999)
            slope = column.max() * column.max()
    
    # When x-values and y-values are not 0
    else:
        
        # Use standard slope formula
        slope = (b[1] - a[1])/(b[0]-a[0])
    
    
    # Equation for y-intercept (solving y=mx+b for b)
    y_int = a[1] - slope * a[0]
    
    # Return slope and y-intercept
    return slope, y_int

In [224]:
def manhattan_cols(df):
    
    upper_right = (-73.929224, 40.804328)
    bottom_right = (-73.980036, 40.710706)
    bottom_left = (-74.054880, 40.681292)
    upper_left = (-73.966303, 40.830050)

    m_top, b_top = two_points_line(upper_right, upper_left, df.pickup_latitude)
    m_left, b_left = two_points_line(bottom_left, upper_left, df.pickup_latitude)
    m_right, b_right = two_points_line(bottom_right, upper_right, df.pickup_latitude)
    m_bottom, b_bottom = two_points_line(bottom_right, bottom_left, df.pickup_latitude)

    def manhattan(row):
        if (((row['pickup_latitude'] <= (row['pickup_longitude'] * m_top + b_top)) &
        (row['pickup_latitude'] >= (row['pickup_longitude'] * m_bottom + b_bottom))) &
        ((row['pickup_latitude'] >= (row['pickup_longitude'] * m_right + b_right)) &
        (row['pickup_latitude'] <= (row['pickup_longitude'] * m_left + b_left)))) & (((row['dropoff_latitude'] <= (row['dropoff_longitude'] * m_top + b_top)) &
        (row['dropoff_latitude'] >= (row['dropoff_longitude'] * m_bottom + b_bottom))) &
        ((row['dropoff_latitude'] >= (row['dropoff_longitude'] * m_right + b_right)) &
        (row['dropoff_latitude'] <= (row['dropoff_longitude'] * m_left + b_left)))):
            return 1
        else:
            return 0
        
    df['manhattan'] = df.apply(manhattan, axis=1)
        
    return df

In [225]:
def newark_cols(df):
    
    upper_right = (-74.107867, 40.718282)
    bottom_right = (-74.143665, 40.654673)
    bottom_left = (-74.250524, 40.698436)
    upper_left = (-74.171983, 40.792347)

    m_top, b_top = two_points_line(upper_right, upper_left, df.pickup_latitude)
    m_left, b_left = two_points_line(bottom_left, upper_left, df.pickup_latitude)
    m_right, b_right = two_points_line(bottom_right, upper_right, df.pickup_latitude)
    m_bottom, b_bottom = two_points_line(bottom_right, bottom_left, df.pickup_latitude)

    def newark(row):
        if (((row['pickup_latitude'] <= (row['pickup_longitude'] * m_top + b_top)) &
        (row['pickup_latitude'] >= (row['pickup_longitude'] * m_bottom + b_bottom))) &
        ((row['pickup_latitude'] >= (row['pickup_longitude'] * m_right + b_right)) &
        (row['pickup_latitude'] <= (row['pickup_longitude'] * m_left + b_left)))) | (((row['dropoff_latitude'] <= (row['dropoff_longitude'] * m_top + b_top)) &
        (row['dropoff_latitude'] >= (row['dropoff_longitude'] * m_bottom + b_bottom))) &
        ((row['dropoff_latitude'] >= (row['dropoff_longitude'] * m_right + b_right)) &
        (row['dropoff_latitude'] <= (row['dropoff_longitude'] * m_left + b_left)))):
            return 1
        else:
            return 0
        
    df['newark'] = df.apply(newark, axis=1)
    
    return df

In [226]:
def jkf_cols(df):
    
    upper_right = (-73.789700, 40.663781)
    bottom_right = (-73.762112, 40.633567)
    bottom_left = (-73.818920, 40.642250)
    upper_left = (-73.804656, 40.664858)

    m_top, b_top = two_points_line(upper_right, upper_left, df.pickup_latitude)
    m_left, b_left = two_points_line(bottom_left, upper_left, df.pickup_latitude)
    m_right, b_right = two_points_line(bottom_right, upper_right, df.pickup_latitude)
    m_bottom, b_bottom = two_points_line(bottom_right, bottom_left, df.pickup_latitude)

    def jfk(row):
        if (((row['pickup_latitude'] <= (row['pickup_longitude'] * m_top + b_top)) &
        (row['pickup_latitude'] >= (row['pickup_longitude'] * m_bottom + b_bottom))) &
        ((row['pickup_latitude'] <= (row['pickup_longitude'] * m_right + b_right)) &
        (row['pickup_latitude'] <= (row['pickup_longitude'] * m_left + b_left)))) | (((row['dropoff_latitude'] <= (row['dropoff_longitude'] * m_top + b_top)) &
        (row['dropoff_latitude'] >= (row['dropoff_longitude'] * m_bottom + b_bottom))) &
        ((row['dropoff_latitude'] <= (row['dropoff_longitude'] * m_right + b_right)) &
        (row['dropoff_latitude'] <= (row['dropoff_longitude'] * m_left + b_left)))):
            return 1
        else:
            return 0
        
    df['jfk'] = df.apply(jfk, axis=1)
            
    return df

In [227]:
def add_locations(df):
    df = manhattan_cols(df)
    df = jkf_cols(df)
    df = newark_cols(df)
    return df

In [228]:
def add_cols(df):
    df = add_Time(df)
    df = add_locations(df)
    return df

## Choose Columns

In [229]:
def choose_predictor_cols(df, cols=['month', 'year', 'dayofweek', 'total_seconds', '15_min_intervals', 'weekend', 'morning_rush', 'night_charge', 'weekday_surcharge', 'manhattan', 'jfk', 'newark', 'passenger_count','euclidean_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']):
    X = df[cols]
    return X

## Min Max Scaler

In [230]:
def min_max_scaler(X):
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_df = pd.DataFrame(X_scaled, columns=X.columns)
    return X_df

In [239]:
def one_hot_cols(X):
    X = one_Hot_Encoder(X, X['month'])
    X = one_Hot_Encoder(X, X['dayofweek'], month=False)
    return X

In [240]:
def one_Hot_Encoder(X, col, month=True): 
    encoder = OneHotEncoder()
    col = encoder.fit_transform(np.array(col).reshape(-1,1))
    col = col.toarray()
    hot_df = pd.DataFrame(col)
    if month:
        hot_df.columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    else:
        hot_df.columns = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
    new_df = X.join(hot_df)
    return new_df

## Pipeline

In [232]:
def my_pipeline(test_set=False):
    
    if test_set:
        df = file_to_dataFrame('test.csv')
        X, y = make_Xtest_ytest(df)
        X = add_distance(X)
    
    else:
        df = file_to_dataFrame('train.csv')
        df = row_elimination(df)
        X, y = make_X_y(df)
    
    X = add_cols(X)
    X = choose_predictor_cols(X)
    X = min_max_scaler(X)
    X = one_Hot_Encoder(X)
    
    return X, y

In [237]:
X_train, y_train = my_pipeline()
X_test, y_test = my_pipeline(test_set=True)

dropping nan: 1000000
nan dropped: 999990
US Mainland Only dropped: 979520
NYC Taxis Only: 978430
Max Passengers 7: 974949
Distance Columns added...
Min fares dropped: 946884
Max fares dropped: 946364
No distance dropped: 936346
Distance Columns added...


## ML Tests

In [130]:
def predict_y(model, X_test):
    y_pred = model.predict(X_test)
    return y_pred

In [154]:
def submit_to_kaggle(y_pred):    
    y_test['fare_amount'] = y_pred
    y_test.to_csv('my_submission.csv', index=False)
    print(y_test)

### Linear Regression

In [155]:
def lin_reg_test(X_train, y_train, X_test):
        
    print('Length of X:', len(X_train))
    clf = LinearRegression()
    clf.fit(X_train, y_train)
    scores = cross_val_score(clf, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
    rmse = np.sqrt(-scores)
    print('Lin reg train rmse:', rmse)
    print('Lin reg train mean:', rmse.mean())
    print('Lin reg train std:', rmse.std())
    
    y_pred = model.predict(X_test)
    submit_to_kaggle(y_pred)
    
    return clf

In [158]:
lin_reg_model = lin_reg_test(X_train, y_train, X_test)

Length of X: 140311
Lin reg train rmse: [3.53290023 3.72230434 3.73733944 3.54865963 3.61891226]
Lin reg train mean: 3.6320231786015547
Lin reg train std: 0.08507519076721042
                                key  fare_amount
0       2015-01-27 13:08:24.0000002    10.544257
1       2015-01-27 13:08:24.0000003    10.795924
2       2011-10-08 11:53:44.0000002     6.068465
3       2012-12-01 21:12:12.0000002     8.237116
4       2012-12-01 21:12:12.0000003    13.988656
5       2012-12-01 21:12:12.0000005    10.424865
6       2011-10-06 12:10:20.0000001     7.426294
7       2011-10-06 12:10:20.0000003    48.532370
8       2011-10-06 12:10:20.0000002    12.020170
9       2014-02-18 15:22:20.0000002     8.352579
10      2014-02-18 15:22:20.0000003    10.321574
11      2014-02-18 15:22:20.0000001    15.121286
12      2010-03-29 20:20:32.0000002     4.802007
13      2010-03-29 20:20:32.0000001     7.335894
14      2011-10-06 03:59:12.0000002     8.186905
15      2011-10-06 03:59:12.0000001    12

### Deep Learning (Sequential)

In [169]:
# keras_regression_test requires "from sklearn.model_selection import train_test_split"
def keras_regression_test(X_train, y_train, X_test, numbers=[128,64], batch_size=32, activation='relu', optimizer='adam', loss='mean_squared_error'):
        
    #X_train, X_check, y_train, y_check = train_test_split(X, y)
    
    # Save the number of columns in predictors: n_cols
    n_cols = X_train.shape[1]

    # Set up the model: model
    model = Sequential()
    
    # Add the first layer
    model.add(Dense(numbers[0], activation=activation, input_shape=(n_cols,)))
    
    # Add addition layers
    for i in range(len(numbers)-1):
        model.add(Dense(numbers[i+1], activation=activation))

    # Add the output layer
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer=optimizer, loss=loss)

    # Define early_stopping_monitor
    early_stopping_monitor = EarlyStopping(patience=2)

    # Fit the model
    model.fit(X_train, y_train, validation_split=0.05, epochs=30, batch_size=batch_size, callbacks=[early_stopping_monitor])

    # Get score for predictions
    #score = model.evaluate(X_check, y_check)
    
    # Get root mean squared error
    #rmse = np.sqrt(score)
    
    # Return root mean squared error
    #print(rmse)
    
    y_pred = model.predict(X_test)

    submit_to_kaggle(y_pred)
    
    return model

In [170]:
keras_model = keras_regression_test(X_train, y_train, X_test, numbers=[150, 150, 20])

Train on 133295 samples, validate on 7016 samples
Epoch 1/30
133295/133295 [==============================] - 6s 42us/step - loss: 18.4612 - val_loss: 12.4270
Epoch 2/30
133295/133295 [==============================] - 5s 39us/step - loss: 13.2583 - val_loss: 11.3641
Epoch 3/30
133295/133295 [==============================] - 5s 39us/step - loss: 12.9038 - val_loss: 12.8167
Epoch 4/30
133295/133295 [==============================] - 6s 43us/step - loss: 12.6240 - val_loss: 11.2939
Epoch 5/30
133295/133295 [==============================] - 5s 39us/step - loss: 12.3720 - val_loss: 11.4262
Epoch 6/30
133295/133295 [==============================] - 5s 40us/step - loss: 12.1844 - val_loss: 11.2615
Epoch 7/30
133295/133295 [==============================] - 5s 40us/step - loss: 12.0647 - val_loss: 12.0369
Epoch 8/30
133295/133295 [==============================] - 5s 41us/step - loss: 11.8806 - val_loss: 10.8719
Epoch 9/30
133295/133295 [==============================] - 6s 43us/step - los

### Random Forests

In [55]:
def random_forest_tuner(X,y):
        
    param_grid = [
        {'n_estimators': [50, 60, 70], 'max_features': [9, 10, 11]}, 
    ]
    
    forest_reg = RandomForestRegressor()
    
    forest_reg_tuned = GridSearchCV(forest_reg, param_grid, cv=3, 
                                    scoring='neg_mean_squared_error')
    
    forest_reg_tuned.fit(X,y)
    
    # Print the tuned parameters and score
    print("Tuned Random Forest Parameters: {}".format(forest_reg_tuned.best_params_))
    
    scores = cross_val_score(forest_reg_tuned, X, y, scoring='neg_mean_squared_error', cv=5)
    
    display_scores('Random Forest', scores)
    
    return forest_reg_tuned

In [45]:
def display_scores(title, scores):
    rmse = np.sqrt(-scores)
    print(title, ' rmse scores:', rmse)
    print(title, ' mean score:', rmse.mean())
    print(title, ' std:', rmse.std())

In [ ]:
def random_forest(X, y, X_test):
    
    forest_reg = RandomForestRegressor(max_features=10, n_estimators=40)
    
    forest_reg.fit(X,y)
    
    scores = cross_val_score(forest_reg, X, y, scoring='neg_mean_squared_error', cv=5)
    
    display_scores('Random Forest', scores)
    
    y_pred = forest_reg.predict(X_test)
    submit_to_kaggle(y_pred)
        
    return forest_reg

In [ ]:
forest_reg = random_forest(X_train, y_train, X_test)
feature_importances = forest_reg.feature_importances_
sorted(zip(feature_importances, list(X_train)), reverse=True)

In [651]:
# cuda
# p42xlarge
# ec2 instance pricing
# make sure you have gpu and optimization
# save as pickle file